# Basig usage of qmrExchange

### Import required libraries

In [1]:
from source.qmr_exchange import Exchange
from source.qmr_exchange import Exchange, Simulator
from source.Agents import RandomMarketTaker, NaiveMarketMaker
from datetime import datetime

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
from_date = datetime(2022,1,1)
to_date = datetime(2022,1,15)
time_interval = 'minute'
tickers = ['XYZ']

### Instantiate a Simulator

In [3]:
sim = Simulator(from_date, to_date,time_interval)
sim.exchange.create_asset(tickers[0])

### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = NaiveMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = RandomMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [11]:
print(sim.trades.head(20).to_markdown())

[<Trade: XYZ 1@100 2022-01-01 00:00:00>, <Trade: XYZ 0@99.75 2022-01-01 00:05:00>, <Trade: XYZ 0@99.5 2022-01-01 00:10:00>, <Trade: XYZ 1@99.75 2022-01-01 00:12:00>, <Trade: XYZ 1@99.5 2022-01-01 00:16:00>, <Trade: XYZ 1@99.75 2022-01-01 00:20:00>, <Trade: XYZ 1@99.5 2022-01-01 00:22:00>, <Trade: XYZ 0@99.25 2022-01-01 00:25:00>, <Trade: XYZ 1@99.5 2022-01-01 00:28:00>, <Trade: XYZ 1@99.25 2022-01-01 00:29:00>, <Trade: XYZ 0@99.0 2022-01-01 00:33:00>, <Trade: XYZ 1@99.25 2022-01-01 00:34:00>, <Trade: XYZ 1@99.5 2022-01-01 00:36:00>, <Trade: XYZ 2@99.25 2022-01-01 00:39:00>, <Trade: XYZ 0@99.0 2022-01-01 00:43:00>, <Trade: XYZ 0@99.0 2022-01-01 00:46:00>, <Trade: XYZ 0@99.0 2022-01-01 00:47:00>, <Trade: XYZ 0@99.0 2022-01-01 00:48:00>, <Trade: XYZ 0@99.0 2022-01-01 00:52:00>, <Trade: XYZ 1@99.25 2022-01-01 00:56:00>, <Trade: XYZ 1@99.5 2022-01-01 00:57:00>, <Trade: XYZ 2@99.25 2022-01-01 01:01:00>, <Trade: XYZ 1@99.5 2022-01-01 01:03:00>, <Trade: XYZ 1@99.25 2022-01-01 01:04:00>, <Trade

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

| dt                  |   open |   high |   low |   close |   volume |
|:--------------------|-------:|-------:|------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 100    | 99.5  |   99.75 |        2 |
| 2022-01-01 00:15:00 |  99.5  |  99.75 | 99.25 |   99.25 |        5 |
| 2022-01-01 00:30:00 |  99    |  99.5  | 99    |   99    |        4 |
| 2022-01-01 00:45:00 |  99    |  99.5  | 99    |   99.5  |        2 |
| 2022-01-01 01:00:00 |  99.25 |  99.5  | 99    |   99.25 |        7 |
| 2022-01-01 01:15:00 |  99    |  99.25 | 98.5  |   98.5  |        5 |
| 2022-01-01 01:30:00 |  98.25 |  98.75 | 98    |   98.74 |        7 |
| 2022-01-01 01:45:00 |  98.49 |  98.73 | 97.99 |   98.48 |        9 |
| 2022-01-01 02:00:00 |  98.73 |  98.98 | 98.23 |   98.23 |        6 |
| 2022-01-01 02:15:00 |  97.98 |  97.98 | 97.26 |   97.26 |        2 |
| 2022-01-01 02:30:00 |  97.02 |  97.5  | 97.02 |   97.26 |        8 |
| 2022-01-01 02:45:00 |  97.5  |  97.98 | 97.5  |   97.98 |        8 |
| 2022

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

|                     |   XYZ |   cash |   aum |
|:--------------------|------:|-------:|------:|
| 2022-01-01 00:00:00 |  0    |   0    |  0    |
| 2022-01-01 00:01:00 |  0    |   0    |  0    |
| 2022-01-01 00:02:00 |  0    |   0    |  0    |
| 2022-01-01 00:03:00 |  0    |   0    |  0    |
| 2022-01-01 00:04:00 |  0    |   0    |  0    |
| 2022-01-01 00:05:00 |  0    |   0    |  0    |
| 2022-01-01 00:06:00 |  0    |   0    |  0    |
| 2022-01-01 00:07:00 |  0    |   0    |  0    |
| 2022-01-01 00:08:00 |  0    |   0    |  0    |
| 2022-01-01 00:09:00 |  0    |   0    |  0    |
| 2022-01-01 00:10:00 |  0    |   0    |  0    |
| 2022-01-01 00:11:00 |  0    |   0    |  0    |
| 2022-01-01 00:12:00 | 99.75 | -99.75 |  0    |
| 2022-01-01 00:13:00 | 99.75 | -99.75 |  0    |
| 2022-01-01 00:14:00 | 99.75 | -99.75 |  0    |
| 2022-01-01 00:15:00 | 99.75 | -99.75 |  0    |
| 2022-01-01 00:16:00 |  0    |  -0.25 | -0.25 |
| 2022-01-01 00:17:00 |  0    |  -0.25 | -0.25 |
| 2022-01-01 00:18:0

### Plot the results

In [9]:
from source.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

In [10]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()